<a href="https://colab.research.google.com/github/ykitaguchi77/GO_AI_project/blob/main/make_super_extend_dataset_for_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Create extended dataset for YOLOv5 implementation**

#Olympia
・Olympia dataset

Treated 640px: 561枚。テストに50枚を残す（test_olympia）、残りの511枚をtrain/valに回す

Control 640px: 169枚。テストに50枚を残す(test_control)、残りの119枚をtrain/valに回す

#Handai
・GO_extended_dataset

control: 1886枚。テストに50枚を残す(test_handai)、残りの1836枚をtrain/valに回す

treatable: 1879枚。テストに50枚を残す(test_handai)、残りの1829枚をtrain/valに回す


#合計：
treatable: 1829 + 119

control: 1836 + 511

testset: Olympia treated 50, control 50, Handai treated 50, control 50

##既存のYOLOでbounding boxを作成 --> classは既知のもので。Glaucomaのrepositoryを参考にしてYOLO datasetを作成。

In [1]:
import pandas as pd
import os
import glob
import shutil
import glob
import random
from PIL import Image
import time
%matplotlib inline

#サポートパッチのインポート
from google.colab.patches import cv2_imshow
import cv2


# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#ファイル数確認用
import os

def countfile(name, path):
    print(f"path: {path}")
    print(f"number: {len(os.listdir(path))}")
    print("")

if not os.path.exists("/content/GO_extended_dataset"):
    !unzip /content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_extended_dataset.zip -d /content/


olympia_cont_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/control_640px"
olympia_grav_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/treated_640px"
handai_cont_dir = "/content/GO_extended_dataset/Control_photo_1886mai"
handai_grav_dir = "/content/GO_extended_dataset/treatable"


countfile("control_640px_olympia", olympia_cont_dir)
countfile("treated_640px_olympia", olympia_grav_dir)

countfile("control_handai", handai_cont_dir)
countfile("treated_handai", handai_grav_dir)


Archive:  /content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_extended_dataset.zip
  inflating: /content/GO_extended_dataset/Control_photo_1886mai/1000.jpg  
  inflating: /content/GO_extended_dataset/Control_photo_1886mai/1010.jpg  
  inflating: /content/GO_extended_dataset/Control_photo_1886mai/1012.jpg  
  inflating: /content/GO_extended_dataset/Control_photo_1886mai/1017.jpg  
  inflating: /content/GO_extended_dataset/Control_photo_1886mai/1019.jpg  
  inflating: /content/GO_extended_dataset/Control_photo_1886mai/1021.jpg  
  inflating: /content/GO_extended_dataset/Control_photo_1886mai/1023.jpg  
  inflating: /content/GO_extended_dataset/Control_photo_1886mai/1025.jpg  
  inflating: /content/GO_extended_dataset/Control_photo_1886mai/1029.jpg  
  inflating: /content/GO_extended_dataset/Control_photo_1886mai/1031.jpg  
  inflating: /content/GO_extended_dataset/Control_photo_1886mai/1033.JPG  
  inflating: /content/GO_extended_dataset/Control_photo_1886mai/1034.jpg  
  inflati

In [ ]:
dst_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO"

if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)
os.makedirs(dst_dir)
os.makedirs(f"{dst_dir}/train_val")
os.makedirs(f"{dst_dir}/train_val/grav")
os.makedirs(f"{dst_dir}/train_val/cont")
os.makedirs(f"{dst_dir}/test")
os.makedirs(f"{dst_dir}/test/grav")
os.makedirs(f"{dst_dir}/test/cont")

In [ ]:
################
# olympia dataset
################

#^^^^^^^^^^^^
# testset作成
#^^^^^^^^^^^^

import os
import shutil
import random

random_state = 1

# Control images
src_cont = olympia_cont_dir
test_cont = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/cont"
trainval_cont = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont"

cont_images = os.listdir(src_cont)
random.Random(random_state).shuffle(cont_images)

test_cont_images = cont_images[:69]
trainval_cont_images = cont_images[69:]

for image in test_cont_images:
    src_path = os.path.join(src_cont, image)
    dst_path = os.path.join(test_cont, image)
    shutil.copy(src_path, dst_path)

for image in trainval_cont_images:
    src_path = os.path.join(src_cont, image)
    dst_path = os.path.join(trainval_cont, image)
    shutil.copy(src_path, dst_path)


# Grav images
src_grav = olympia_grav_dir
test_grav = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/grav"
trainval_grav = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav"

grav_images = os.listdir(src_grav)
random.Random(random_state).shuffle(grav_images)

test_grav_images = grav_images[:69]
trainval_grav_images = grav_images[69:]

for image in test_grav_images:
    src_path = os.path.join(src_grav, image)
    dst_path = os.path.join(test_grav, image)
    shutil.copy(src_path, dst_path)

for image in trainval_grav_images:
    src_path = os.path.join(src_grav, image)
    dst_path = os.path.join(trainval_grav, image)
    shutil.copy(src_path, dst_path)





In [ ]:
import os
from shutil import copyfile
from tqdm import tqdm

grav_dir = olympia_grav_dir
cont_dir = olympia_cont_dir
test_grav_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/grav"
test_cont_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/cont"
train_val_grav_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav"
train_val_cont_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont"

# Check if files in grav_dir are in test_grav_dir
for filename in tqdm(os.listdir(grav_dir)):
    basename = os.path.basename(filename)
    if os.path.exists(os.path.join(test_grav_dir, basename)):
        print("coooool!!!")
        continue
    else:
        copyfile(os.path.join(grav_dir, filename), os.path.join(train_val_grav_dir, filename))

# Check if files in cont_dir are in test_cont_dir
for filename in tqdm(os.listdir(cont_dir)):
    basename = os.path.basename(filename)
    if os.path.exists(os.path.join(test_cont_dir, basename)):
        print("hotttttt!!!")
        continue
    else:
        copyfile(os.path.join(cont_dir, filename), os.path.join(train_val_cont_dir, filename))


In [ ]:
#####################
# rename
#指定されたフォルダ内にあるファイル名が"olympia_"から始まらない場合に、"olympia_"を先頭につけたファイル名にリネームする処理を行う。
#####################

import os

# フォルダのパスを指定
test_grav_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/grav"
test_cont_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/cont"
train_val_grav_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav"
train_val_cont_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont"

# ファイル名が "olympia_" から始まらない場合、"olympia_" を先頭につけたファイル名にリネームする
def rename_file(path):
    for filename in os.listdir(path):
        if not filename.startswith("oly_"):
            new_filename = "oly_" + filename
            os.rename(os.path.join(path, filename), os.path.join(path, new_filename))

# リネーム処理を実行
rename_file(test_grav_dir)
rename_file(test_cont_dir)
rename_file(train_val_grav_dir)
rename_file(train_val_cont_dir)


In [ ]:
print(len(os.listdir("/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav")))

2260


In [ ]:
############################
# Handai GO_extended_dataset
#
# /treatable → /train_val/grav
# /control → /train_val/Control_photo_1886mai
#
############################

!unzip /content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_extended_dataset.zip -d /content/
!cp /content/GO_extended_dataset/Control_photo_1886mai/* /content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont/
!cp /content/GO_extended_dataset/treatable/* /content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav/
